# Align the Beaded Helix using shapeGMMTorch libraries

In this notebook I will demonstrate how to perform a single maximum likelihood alignment of a trajectory using the shapeGMMTorch libraries.  I will do this for both uniform and kronecker covariance models.

I will use MDAnalysis to read the trajectory but anyways of reading a trajectory and generating a `(n_frames, n_atoms, 3)` array is fine.

## Import Libraries

In [6]:
import sys
import numpy as np
# we will use MDAnalysis to read trajectory
import MDAnalysis as md
# the alignment routines are contained in torch_align
from shapeGMMTorch import torch_align
# we will also need to load pytorch to prepare the data
import torch

## Read trajectory

In [20]:
prmtopFileName = "../beaded_helix_example/helix_template.pdb"
trajFileName = "../beaded_helix_example/helix_folding_eps6.0.dcd"
coord = md.Universe(prmtopFileName,trajFileName)
print("Number of atoms in trajectory:", coord.atoms.n_atoms)
print("Number of frames in trajectory:", coord.trajectory.n_frames)
# make atom selection
atomSel = coord.select_atoms('all')
print("Number of atoms in selection:", atomSel.n_atoms)
# create traj data of selection
traj_numpy = np.empty((coord.trajectory.n_frames,atomSel.n_atoms,3),dtype=float)
#loop traj
for ts in coord.trajectory:
    traj_numpy[ts.frame,:] = atomSel.positions

/Users/mmccull/opt/anaconda3/envs/nomkl/lib/python3.10/site-packages/MDAnalysis/topology/PDBParser.py:317: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "
/Users/mmccull/opt/anaconda3/envs/nomkl/lib/python3.10/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: 1
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/Users/mmccull/opt/anaconda3/envs/nomkl/lib/python3.10/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: 10
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/Users/mmccull/opt/anaconda3/envs/nomkl/lib/python3.10/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed 

Number of atoms in trajectory: 12
Number of frames in trajectory: 10001
Number of atoms in selection: 12


## Convert trajectory to a torch tensor and substract center of geometry

In [10]:
# for torch we need to declare the device to perform operation
device = torch.device("cpu")
dtype = torch.float32

In [12]:
# create torch tensor
traj_tensor = torch.tensor(traj_numpy,device=device,dtype=dtype)
# remove COG
torch_align.torch_remove_center_of_geometry(traj_tensor)

## Uniform alignment

Below I will perform a uniform alignment.  The alignment routine takes in and returns torch tensors.  The routine returns the aligned positions, average, and variance.  If you put `verbose=True` the routine will print the log likelihood of each iteration.

In [16]:
uniform_aligned_traj_tensor, uniform_avg_tensor, uniform_var_tensor = torch_align.torch_iterative_align_uniform(traj_tensor, device=device, dtype=dtype, verbose=True)

14.103167715892111
15.653984409911542
15.699899158017994
15.716427872375503
15.725605117749549
15.730752390271883
15.733637576830931
15.735245252710037
15.736138815441734


In [18]:
# pass tensor back to cpu/numpy
uniform_aligned_traj_numpy = uniform_aligned_traj_tensor.cpu().numpy()

## Kronecker alignment

In [17]:
kronecker_aligned_traj_tensor, kronecker_avg_tensor, kronecker_precision_tensor, kronecker_lpdet_tensor = torch_align.torch_iterative_align_kronecker(traj_tensor, device=device, dtype=dtype, verbose=True)

tensor(32.0833, dtype=torch.float64)
tensor(34.5350, dtype=torch.float64)
tensor(34.8472, dtype=torch.float64)
tensor(35.0027, dtype=torch.float64)
tensor(35.1261, dtype=torch.float64)
tensor(35.2426, dtype=torch.float64)
tensor(35.3564, dtype=torch.float64)
tensor(35.4575, dtype=torch.float64)
tensor(35.5334, dtype=torch.float64)
tensor(35.5858, dtype=torch.float64)
tensor(35.6198, dtype=torch.float64)
tensor(35.6408, dtype=torch.float64)
tensor(35.6536, dtype=torch.float64)
tensor(35.6612, dtype=torch.float64)
tensor(35.6659, dtype=torch.float64)
tensor(35.6687, dtype=torch.float64)
tensor(35.6705, dtype=torch.float64)
tensor(35.6717, dtype=torch.float64)
tensor(35.6725, dtype=torch.float64)


In [19]:
kronecker_aligned_traj_numpy = kronecker_aligned_traj_tensor.cpu().numpy()